In [2]:
import os
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

API_KEY = open('key.txt', 'r').read().strip()
os.environ["OPENAI_API_KEY"] = API_KEY

from dotenv import load_dotenv
load_dotenv()

False

In [14]:
host="localhost"
port="5432"
database="ReportDB"
user="postgres"
password="postgres"

db = SQLDatabase.from_uri(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")

llm = ChatOpenAI(model_name="gpt-4", temperature=0)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [15]:
SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.
"""

SQL_FUNCTIONS_SUFFIX = """I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables."""

FORMAT_INSTRUCTIONS = """Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question"""

In [30]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    prefix=SQL_PREFIX,
    suffix=SQL_FUNCTIONS_SUFFIX,
    format_instructions=FORMAT_INSTRUCTIONS,
    # return_full_results=True,
    # return_intermediate_results=True,
    agent_executor_kwargs = {'return_intermediate_steps': True}
)

In [31]:
steps = agent_executor("Who are the top 5 retailers for the month of May?")



> Entering new  chain...

Invoking: `sql_db_list_tables` with ``


monthWise
Invoking: `sql_db_schema` with `monthWise`



CREATE TABLE "monthWise" (
	"Date" DATE, 
	"Retail ID" TEXT, 
	"Retail Name" TEXT, 
	"Zone" TEXT, 
	"Location" TEXT, 
	"Total Play time" INTERVAL, 
	"Total Play time Minutes" INTERVAL, 
	"Efficiency %%" DOUBLE PRECISION, 
	"Start Time" TIMESTAMP WITHOUT TIME ZONE, 
	"End Time" TIMESTAMP WITHOUT TIME ZONE, 
	"Count Device Offline time (hours)" INTERVAL, 
	"Remarks" TEXT
)

/*
3 rows from monthWise table:
Date	Retail ID	Retail Name	Zone	Location	Total Play time	Total Play time Minutes	Efficiency %	Start Time	End Time	Count Device Offline time (hours)	Remarks
2023-03-01	Basic LED TV	GRD-004 Soudia Store	New Market Zone (75 Screens)	Green Road	12:10:35	0:12:11	122.0	2023-03-01 10:27:27	2023-03-01 23:19:36	0:22:00	None
2023-03-01	HAMIM 24"	GRD-007 Ma genarel store	New Market Zone (75 Screens)	Green Road	11:52:08	0:11:52	119.0	2023-03-01 09:55:49	2023-03-01 22:05:03	0:

In [32]:
print(steps.keys())

dict_keys(['input', 'output', 'intermediate_steps'])


In [33]:
steps['intermediate_steps']

[(_FunctionsAgentAction(tool='sql_db_list_tables', tool_input='', log='\nInvoking: `sql_db_list_tables` with ``\n\n\n', message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'sql_db_list_tables', 'arguments': '{\n  "__arg1": ""\n}'}}, example=False)]),
  'monthWise'),
 (_FunctionsAgentAction(tool='sql_db_schema', tool_input='monthWise', log='\nInvoking: `sql_db_schema` with `monthWise`\n\n\n', message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'sql_db_schema', 'arguments': '{\n  "__arg1": "monthWise"\n}'}}, example=False)]),
  '\nCREATE TABLE "monthWise" (\n\t"Date" DATE, \n\t"Retail ID" TEXT, \n\t"Retail Name" TEXT, \n\t"Zone" TEXT, \n\t"Location" TEXT, \n\t"Total Play time" INTERVAL, \n\t"Total Play time Minutes" INTERVAL, \n\t"Efficiency %%" DOUBLE PRECISION, \n\t"Start Time" TIMESTAMP WITHOUT TIME ZONE, \n\t"End Time" TIMESTAMP WITHOUT TIME ZONE, \n\t"Count Device Offline time (hours)" INTERVAL, \n\t"Remarks" TEXT\n)\n\n/*\n3 r

In [88]:
for step in steps['intermediate_steps']:
    print(step[0].tool)
    print(step[0].tool_input)
    print(step[0].log)
    print(step[0].message_log[0].content)
    print(step[1])

sql_db_list_tables


Invoking: `sql_db_list_tables` with ``




monthWise
sql_db_schema
monthWise

Invoking: `sql_db_schema` with `monthWise`





CREATE TABLE "monthWise" (
	"Date" DATE, 
	"Retail ID" TEXT, 
	"Retail Name" TEXT, 
	"Zone" TEXT, 
	"Location" TEXT, 
	"Total Play time" INTERVAL, 
	"Total Play time Minutes" INTERVAL, 
	"Efficiency %%" DOUBLE PRECISION, 
	"Start Time" TIMESTAMP WITHOUT TIME ZONE, 
	"End Time" TIMESTAMP WITHOUT TIME ZONE, 
	"Count Device Offline time (hours)" INTERVAL, 
	"Remarks" TEXT
)

/*
3 rows from monthWise table:
Date	Retail ID	Retail Name	Zone	Location	Total Play time	Total Play time Minutes	Efficiency %	Start Time	End Time	Count Device Offline time (hours)	Remarks
2023-03-01	Basic LED TV	GRD-004 Soudia Store	New Market Zone (75 Screens)	Green Road	12:10:35	0:12:11	122.0	2023-03-01 10:27:27	2023-03-01 23:19:36	0:22:00	None
2023-03-01	HAMIM 24"	GRD-007 Ma genarel store	New Market Zone (75 Screens)	Green Road	11:52:08	0:11:52	119.0	2023-03-01 09:55:49	

In [73]:
print(steps['intermediate_steps'][-1][0].tool)

sql_db_query


In [75]:
print(steps['intermediate_steps'][-1][0].tool_input)

SELECT "Retail Name", SUM("Total Play time Minutes") as TotalTime FROM "monthWise" WHERE EXTRACT(MONTH FROM "Date") = 5 GROUP BY "Retail Name" ORDER BY TotalTime DESC LIMIT 5


In [76]:
print(steps['intermediate_steps'][-1][0].log)


Invoking: `sql_db_query` with `SELECT "Retail Name", SUM("Total Play time Minutes") as TotalTime FROM "monthWise" WHERE EXTRACT(MONTH FROM "Date") = 5 GROUP BY "Retail Name" ORDER BY TotalTime DESC LIMIT 5`





In [78]:
print(steps['intermediate_steps'][-1][0].message_log[0].content)

In [82]:
print(steps['intermediate_steps'][-1][1])

[('MDK-002 Hakim store', datetime.timedelta(seconds=26521)), ('HBH-004 New General Balaka Store', datetime.timedelta(seconds=26160)), ('ZGT-016 Brothers genarel store', datetime.timedelta(seconds=25634)), ('GRD-004 Soudia Store', datetime.timedelta(seconds=24905)), ('TLP-003 Ma jononi store', datetime.timedelta(seconds=24751))]


In [83]:
steps['input']

'Who are the top 5 retailers for the month of May?'

In [85]:
print(steps['output'])

The top 5 retailers for the month of May are:

1. MDK-002 Hakim store
2. HBH-004 New General Balaka Store
3. ZGT-016 Brothers genarel store
4. GRD-004 Soudia Store
5. TLP-003 Ma jononi store
